In [1]:
import networkx as nx
import pandas as pd

from fairpair import *

In [2]:
def plot_ranking(G:FairPairGraph, ranking:dict):
    ranking = {node:rank for node, rank in sorted(ranking.items(), key=lambda x:x[1], reverse=True)}
    data = []
    for node, rank in ranking.items():
        data.append((G.nodes(data='minority')[node], G.nodes(data='score')[node], rank))
    return pd.DataFrame(data, columns=['Protected', 'Score', 'Rank'])

def plot_ranking_short(G:FairPairGraph, ranking:dict):
    ranking = {node:rank for node, rank in sorted(ranking.items(), key=lambda x:x[1], reverse=True)}
    data = []
    for node in ranking.keys():
        data.append('M' if G.nodes(data='minority')[node] else '_')
    return ' '.join(data)

def evaluate_tau(G:FairPairGraph, ranking:dict):
    tau_min = weighted_tau(G, ranking, G.minority)
    tau_maj = weighted_tau(G, ranking, G.majority)
    return f'protected: {tau_min:.2f}, unprotected: {tau_maj:.2f}, difference: {tau_min - tau_maj:.2f}, ratio: {tau_min/tau_maj if tau_maj>0 else 0:.2f}'

def create_graph_ranking(len:int):
    G = FairPairGraph()
    G.add_nodes_from(range(len))
    scores = {n:(len-n) for n in range(len)}
    nx.function.set_node_attributes(G, scores, 'score')
    labels = {node:False for node in G.nodes}
    nx.function.set_node_attributes(G, labels, 'minority')
    ranking = {n:(len-n) for n in range(len)}
    return G, ranking

## 1: Distinguishability of Groups

In [3]:
G, ranking = create_graph_ranking(8)
nx.function.set_node_attributes(G, {2:True, 5:True}, 'minority')
print('original ranking: ', plot_ranking_short(G, ranking))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

ranking2 = {node:(rank-2 if G.nodes(data='minority')[node] else rank) for node, rank in ranking.items()}
print('with disadvantage:', plot_ranking_short(G, ranking2))
print('Kemedy distances: ', evaluate_tau(G, ranking2), '\n')

ranking3 = {node:(rank+2 if G.nodes(data='minority')[node] else rank) for node, rank in ranking.items()}
print('with advantage:   ', plot_ranking_short(G, ranking3))
print('Kemedy distances: ', evaluate_tau(G, ranking3), '\n')

original ranking:  _ _ M _ _ M _ _
Kemedy distances:  protected: 0.00, unprotected: 0.00, difference: 0.00, ratio: 0.00 

with disadvantage: _ _ _ M _ _ M _
Kemedy distances:  protected: 0.29, unprotected: 0.17, difference: 0.11, ratio: 1.65 

with advantage:    _ M _ _ M _ _ _
Kemedy distances:  protected: 0.29, unprotected: 0.17, difference: 0.11, ratio: 1.65 



    Considering "disadvantage" as a lower rank than would be appropriate given the ground-truth scores and vice versa. We thus look at accuracy and not a the mere position in the ranking. Both difference and ratio fail.

## 2: Boundedness

In [4]:
G, ranking = create_graph_ranking(8)
nx.function.set_node_attributes(G, {0:True}, 'minority')
print('original ranking: ', plot_ranking_short(G, ranking))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

ranking[0] = -1
print('worst case:       ', plot_ranking_short(G, ranking))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

G, ranking = create_graph_ranking(20)
nx.function.set_node_attributes(G, {0:True}, 'minority')
ranking[0] = -1
print('worst case longer:', plot_ranking_short(G, ranking))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

original ranking:  M _ _ _ _ _ _ _
Kemedy distances:  protected: 0.00, unprotected: 0.00, difference: 0.00, ratio: 0.00 

worst case:        _ _ _ _ _ _ _ M
Kemedy distances:  protected: 1.00, unprotected: 0.65, difference: 0.35, ratio: 1.55 

worst case longer: _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ M
Kemedy distances:  protected: 1.00, unprotected: 0.43, difference: 0.57, ratio: 2.32 



    Holds for tau itself because of normalization to (0,1). Therefore holds for difference (in -1,1), but fails for ratio.

## 3: Monotonicity

In [31]:
print('--- equal relevance of protected and unprotected candidate ---')
G, ranking = create_graph_ranking(8)
nx.function.set_node_attributes(G, {2:True, 5:True}, 'minority')
nx.function.set_node_attributes(G, {4:4, 5:4}, 'score')
print('original ranking: ', plot_ranking_short(G, ranking))
print('relevances:       ', ' '.join([str(score) for node, score in G.nodes(data='score')]))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

ranking.update({4:3, 5:4})
print('swap 4 and 5:     ', plot_ranking_short(G, ranking))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

--- equal relevance of protected and unprotected candidate ---
original ranking:  _ _ M _ _ M _ _
relevances:        8 7 6 5 4 4 2 1
Kemedy distances:  protected: 0.00, unprotected: 0.00, difference: 0.00, ratio: 0.00 

swap 4 and 5:      _ _ M _ M _ _ _
Kemedy distances:  protected: 0.00, unprotected: 0.00, difference: 0.00, ratio: 0.00 



In [32]:
print('--- higher relevance of protected candidate ---')
G, ranking = create_graph_ranking(8)
nx.function.set_node_attributes(G, {2:True, 5:True}, 'minority')
nx.function.set_node_attributes(G, {4:4, 5:5}, 'score')
print('original ranking: ', plot_ranking_short(G, ranking))
print('relevances:       ', ' '.join([str(score) for node, score in G.nodes(data='score')]))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

ranking = ranking.copy()
ranking.update({4:3, 5:4})
print('swap 4 and 5:     ', plot_ranking_short(G, ranking))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

--- higher relevance of protected candidate ---
original ranking:  _ _ M _ _ M _ _
relevances:        8 7 6 5 4 5 2 1
Kemedy distances:  protected: 0.10, unprotected: 0.06, difference: 0.05, ratio: 1.85 

swap 4 and 5:      _ _ M _ M _ _ _
Kemedy distances:  protected: 0.00, unprotected: 0.00, difference: 0.00, ratio: 0.00 



    Partially satisfied, because it only holds if swapping two candidated corrects a mistake in the ranking, i.e. if the lower ranked, protected candidate has higher relevance. Does not hold if the lower ranked, protected candidate has equal relevance.

## 4: Deepness

In [33]:
G, ranking = create_graph_ranking(8)
nx.function.set_node_attributes(G, {2:True, 5:True}, 'minority')
print('original ranking: ', plot_ranking_short(G, ranking))
print('relevances:       ', ' '.join([str(score) for node, score in G.nodes(data='score')]))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

ranking2 = ranking.copy()
ranking2.update({4:3, 5:4})
print('swap 4 and 5:     ', plot_ranking_short(G, ranking2))
print('Kemedy distances: ', evaluate_tau(G, ranking2), '\n')

ranking3 = ranking.copy()
ranking3.update({1:6, 2:7})
print('swap 1 and 2:     ', plot_ranking_short(G, ranking3))
print('Kemedy distances: ', evaluate_tau(G, ranking3), '\n')

ranking4 = ranking.copy()
ranking4.update({0:6, 2:8})
print('swap 0 and 2:     ', plot_ranking_short(G, ranking4))
print('Kemedy distances: ', evaluate_tau(G, ranking4), '\n')

original ranking:  _ _ M _ _ M _ _
relevances:        8 7 6 5 4 3 2 1
Kemedy distances:  protected: 0.00, unprotected: 0.00, difference: 0.00, ratio: 0.00 

swap 4 and 5:      _ _ M _ M _ _ _
Kemedy distances:  protected: 0.09, unprotected: 0.06, difference: 0.04, ratio: 1.65 

swap 1 and 2:      _ M _ _ _ M _ _
Kemedy distances:  protected: 0.09, unprotected: 0.06, difference: 0.04, ratio: 1.65 

swap 0 and 2:      M _ _ _ _ M _ _
Kemedy distances:  protected: 0.20, unprotected: 0.14, difference: 0.07, ratio: 1.51 



    Both difference and ratio fail because there is no positional discount. However, swaps between candidates from different groups further away yield higher per-group Kemedy distances and a higher difference (always??). Ratio is not monotonous with the swapping distance.

## 5: Intra-Group Fairness

In [38]:
print('--- swap protected group ---')
G, ranking = create_graph_ranking(8)
nx.function.set_node_attributes(G, {2:True, 5:True}, 'minority')
nx.function.set_node_attributes(G, {2:3, 5:6}, 'score')
print('original ranking: ', plot_ranking_short(G, ranking))
print('relevances:       ', ' '.join([str(score) for node, score in G.nodes(data='score')]))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

ranking.update({2:3, 5:6})
print('swap 2 and 5:     ', plot_ranking_short(G, ranking))
ranking2 = {node:rank for node, rank in sorted(ranking.items(), key=lambda x:x[1], reverse=True)}
print('relevances:       ', ' '.join([str(G.nodes(data='score')[node]) for node, _ in ranking2.items()]))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

--- swap protected group ---
original ranking:  _ _ M _ _ M _ _
relevances:        8 7 3 5 4 6 2 1
Kemedy distances:  protected: 0.48, unprotected: 0.17, difference: 0.31, ratio: 2.77 

swap 2 and 5:      _ _ M _ _ M _ _
relevances:        8 7 6 5 4 3 2 1
Kemedy distances:  protected: 0.00, unprotected: 0.00, difference: 0.00, ratio: 0.00 



In [43]:
print('--- swap unprotected group ---')
G, ranking = create_graph_ranking(8)
nx.function.set_node_attributes(G, {2:True, 5:True}, 'minority')
nx.function.set_node_attributes(G, {1:4, 4:7}, 'score')
print('original ranking: ', plot_ranking_short(G, ranking))
print('relevances:       ', ' '.join([str(score) for node, score in G.nodes(data='score')]))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

ranking.update({1:4, 4:7})
print('swap 1 and 4:     ', plot_ranking_short(G, ranking))
ranking2 = {node:rank for node, rank in sorted(ranking.items(), key=lambda x:x[1], reverse=True)}
print('relevances:       ', ' '.join([str(G.nodes(data='score')[node]) for node, _ in ranking2.items()]))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

--- swap unprotected group ---
original ranking:  _ _ M _ _ M _ _
relevances:        8 4 6 5 7 3 2 1
Kemedy distances:  protected: 0.20, unprotected: 0.24, difference: -0.04, ratio: 0.85 

swap 1 and 4:      _ _ M _ _ M _ _
relevances:        8 7 6 5 4 3 2 1
Kemedy distances:  protected: 0.00, unprotected: 0.00, difference: 0.00, ratio: 0.00 



    Swapping in concordance always improves Kemedy distances. So swapping the protected group improves their accuracy and lowers the difference, and swapping the unprotected group improves their accuracy and increases the difference (satisfied). Does not hold for the ratio, however, as it is lowered either way.

## 6: Invariance to Linear Transformations

In [67]:
G, ranking = create_graph_ranking(4)
nx.function.set_node_attributes(G, {2:True}, 'minority')
print('original ranking: ', plot_ranking_short(G, ranking))
print('relevances:       ', ' '.join([str(score) for node, score in G.nodes(data='score')]))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

scores = {n:(4-n)*2 for n in range(4)}
H = G.copy()
nx.function.set_node_attributes(H, scores, 'score')
print('linear transform: ', plot_ranking_short(H, ranking))
print('relevances:       ', ' '.join([str(score) for node, score in H.nodes(data='score')]))
print('Kemedy distances: ', evaluate_tau(H, ranking), '\n')

rng = np.random.default_rng()
l = list(range(4))
rng.shuffle(l)
ranking = {i:n for i,n in enumerate(l)}
print('shuffled:         ', plot_ranking_short(G, ranking))
ranking2 = {node:rank for node, rank in sorted(ranking.items(), key=lambda x:x[1], reverse=True)}
print('relevances:       ', ' '.join([str(G.nodes(data='score')[node]) for node, _ in ranking2.items()]))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

scores = {n:(4-n)*2 for n in range(4)}
nx.function.set_node_attributes(G, scores, 'score')
print('linear transform: ', plot_ranking_short(G, ranking))
print('relevances:       ', ' '.join([str(score) for node, score in G.nodes(data='score')]))
print('Kemedy distances: ', evaluate_tau(G, ranking), '\n')

original ranking:  _ _ M _
relevances:        4 3 2 1
Kemedy distances:  protected: 0.00, unprotected: 0.00, difference: 0.00, ratio: 0.00 

linear transform:  _ _ M _
relevances:        8 6 4 2
Kemedy distances:  protected: 0.00, unprotected: 0.00, difference: 0.00, ratio: 0.00 

shuffled:          _ M _ _
relevances:        4 2 1 3
Kemedy distances:  protected: 0.41, unprotected: 0.50, difference: -0.09, ratio: 0.82 

linear transform:  _ M _ _
relevances:        8 6 4 2
Kemedy distances:  protected: 0.41, unprotected: 0.50, difference: -0.09, ratio: 0.82 



    Both difference and ratio satisfy the property.